In [1]:
import os
import sys
from sim import simulation as sim
import numpy as np
from netCDF4 import Dataset
import xarray as xr

In [2]:
point = sim.SystemState([1, 2, 3, 4, 5, 6], 0)
traj = sim.Simulator(system_state=point)

In [3]:
integration_time = 1000
integration_steps = int(integration_time/traj.increment)
chunk_steps = 100
chunks = int(integration_steps/chunk_steps)

In [4]:
file_path = '/home/cucchi/phd/prova.nc'

In [5]:
if os.path.exists(file_path):
    os.remove(file_path)

# create netcdf file
"""
dataset = Dataset(file_path, 'w')

node = dataset.createDimension('node', len(point.coords))
time = dataset.createDimension('time', None)

times = dataset.createVariable('time', np.float64, ('time',))
nodes = dataset.createVariable('node', np.int32, ('node',))

var = dataset.createVariable('var', np.float32, ('time', 'node'))
nodes[:] = np.arange(0, len(point.coords))
"""

"\ndataset = Dataset(file_path, 'w')\n\nnode = dataset.createDimension('node', len(point.coords))\ntime = dataset.createDimension('time', None)\n\ntimes = dataset.createVariable('time', np.float64, ('time',))\nnodes = dataset.createVariable('node', np.int32, ('node',))\n\nvar = dataset.createVariable('var', np.float32, ('time', 'node'))\nnodes[:] = np.arange(0, len(point.coords))\n"

In [6]:
point.time

0

In [7]:
with Dataset(file_path, 'w') as dataset:

    node = dataset.createDimension('node', len(point.coords))
    time = dataset.createDimension('time', None)

    times = dataset.createVariable('time', np.float64, ('time',))
    nodes = dataset.createVariable('node', np.int32, ('node',))

    var = dataset.createVariable('var', np.float32, ('time', 'node'))
    nodes[:] = np.arange(0, len(point.coords))

    dim = len(point.coords)

    for chunk in np.arange(0, chunks):
        # initialize chunk numpy array
        t = []
        data_array = np.empty((chunk_steps, dim))
        for i in np.arange(0, chunk_steps):
            data_array[i, :] = point.coords
            t.append(point.time)
            traj.integrate()
        var[(chunk_steps * chunk):(chunk_steps * (chunk + 1)),:] = data_array
        times[(chunk_steps * chunk):(chunk_steps * (chunk + 1))] = t 

In [10]:
sys.getsizeof(data_array)

4912

In [12]:
#dataset.close()

In [8]:
data = xr.open_dataarray(file_path)

In [9]:
data

<xarray.DataArray 'var' (time: 100000, node: 6)>
[600000 values with dtype=float32]
Coordinates:
  * time     (time) float64 0.0 0.01 0.02 0.03 0.04 ... 1e+03 1e+03 1e+03 1e+03
  * node     (node) int32 0 1 2 3 4 5

In [77]:
data.coords['time']

<xarray.DataArray 'time' (time: 1000)>
array([ 2.  ,  2.01,  2.02, ..., 11.97, 11.98, 11.99])
Coordinates:
  * time     (time) float64 2.0 2.01 2.02 2.03 2.04 ... 11.96 11.97 11.98 11.99